In [1]:
%pip install spark-nlp==6.0.2 numpy pandas matplotlib seaborn setuptools wordcloud pypdf2
%pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import findspark
findspark.init('./spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping_id, window, explode, to_json, from_json
from pyspark.sql.functions import udf, lit, current_timestamp, current_date, date_format
import PyPDF2
import os

In [3]:
spark = SparkSession.builder \
    .appName("kafkademo") \
    .config("spark.jars.packages", 
            ",".join([
                "com.johnsnowlabs.nlp:spark-nlp_2.12:6.0.2"
            ])) \
    .getOrCreate()

25/07/27 15:30:15 WARN Utils: Your hostname, Joses-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.105.113.167 instead (on interface en0)
25/07/27 15:30:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/joseaguilar/.ivy2/cache
The jars for the packages stored in: /Users/joseaguilar/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-97fd8a72-3830-4002-a4a7-39bbfaac5d74;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.0.2 in central
	found com.typesafe#config;1.4.2 in central


:: loading settings :: url = jar:file:/Users/joseaguilar/Documents/Development/spark/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in local-m2-cache
	found org.apache.httpcomponents#httpclient;4.5.13 in local-m2-cache
	found org.apache.httpcomponents#httpcore;4.4.13 in local-m2-cache
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#goo

In [4]:
import sparknlp
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer

print(sparknlp.version())
spark = sparknlp.start()

6.0.2


25/07/27 15:30:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
textos_pdf = []
for nombre_archivo in os.listdir("/Users/joseaguilar/Documents/Development/databricks/spark_databricks/pdf/"):
    if nombre_archivo.endswith(".pdf"):
        ruta = os.path.join("pdf", nombre_archivo)
        print(f"Procesando archivo: {ruta}")
        reader = PyPDF2.PdfReader(ruta)
        # Extraer todo el texto concatenando páginas
        texto = ""
        for page in reader.pages:
            texto += page.extract_text() + " "
        textos_pdf.append((nombre_archivo, texto))

# Crear un DataFrame de Spark con columnas 'documento' y 'texto'
df_docs = spark.createDataFrame(textos_pdf, schema=["documento", "texto"])
df_docs.show(1)

Procesando archivo: pdf/2303.10130v5.pdf
Procesando archivo: pdf/1706.03762v7.pdf


+----------------+--------------------+
|       documento|               texto|
+----------------+--------------------+
|2303.10130v5.pdf|WORKING PAPER\nGP...|
+----------------+--------------------+
only showing top 1 row



In [8]:
# Configurar una pipeline de Spark NLP para tokenizar
document_assembler = DocumentAssembler()\
    .setInputCol("linea_texto").setOutputCol("document")
tokenizer = Tokenizer()\
    .setInputCols(["document"]).setOutputCol("token")
# Opcional: normalizador para eliminar puntuación y convertir a minúsculas
normalizer = Normalizer()\
    .setInputCols(["token"]).setOutputCol("token_norm")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\\w¿?¡!áéíóúÁÉÍÓÚüÜñÑ]"])  # quitar símbolos no deseados

finisher = Finisher()\
    .setInputCols(["token_norm"]).setOutputCols(["palabras"])\
    .setCleanAnnotations(False)

# Reutilizar la pipeline de tokenización en la columna 'texto'
from pyspark.ml import Pipeline

document_assembler.setInputCol("texto")
modelo_pdf = Pipeline(stages=[document_assembler, tokenizer, normalizer, finisher]).fit(df_docs)
df_tokens_pdf = modelo_pdf.transform(df_docs).select("documento", "palabras")
df_tokens_pdf.show(1, truncate=100)  # ver tokens de un documento


+----------------+----------------------------------------------------------------------------------------------------+
|       documento|                                                                                            palabras|
+----------------+----------------------------------------------------------------------------------------------------+
|2303.10130v5.pdf|[working, paper, gpts, are, gpts, an, early, look, at, the, labor, market, impact, potential, of,...|
+----------------+----------------------------------------------------------------------------------------------------+
only showing top 1 row



In [8]:
# Explode de la columna de listas de palabras para tener una palabra por fila
from pyspark.sql.functions import explode, col
df_palabras = df_tokens.select(explode(col("palabras")).alias("palabra"))
# Filtrar posibles tokens vacíos o muy cortos si aplica
df_palabras = df_palabras.filter(col("palabra") != "")

# Contar frecuencia de cada palabra
df_freq = df_palabras.groupBy("palabra").count().orderBy(col("count").desc())
df_freq.show(10)

+-------+-----+
|palabra|count|
+-------+-----+
|     de|  346|
|      y|  129|
|     la|  112|
|     en|   91|
|    del|   91|
|  datos|   89|
|   data|   86|
|     el|   75|
|    los|   75|
|      a|   70|
+-------+-----+
only showing top 10 rows



In [9]:
# Explotar las palabras por documento y contar frecuencias dentro de cada documento
df_palabras_pdf = df_tokens_pdf.select(col("documento"), explode(col("palabras")).alias("palabra"))
df_freq_por_doc = df_palabras_pdf.groupBy("documento", "palabra").count()
# Por ejemplo, mostrar las 5 palabras más frecuentes de cada documento
from pyspark.sql.window import Window
import pyspark.sql.functions as F
window = Window.partitionBy("documento").orderBy(F.desc("count"))
df_top5_por_doc = df_freq_por_doc.withColumn("rank", F.row_number().over(window))\
                                 .filter(col("rank") <= 5)\
                                 .orderBy("documento", col("rank"))
df_top5_por_doc.show(20, truncate=False)

+----------------+-------+-----+----+
|documento       |palabra|count|rank|
+----------------+-------+-----+----+
|1706.03762v7.pdf|the    |329  |1   |
|1706.03762v7.pdf|of     |178  |2   |
|1706.03762v7.pdf|and    |167  |3   |
|1706.03762v7.pdf|in     |136  |4   |
|1706.03762v7.pdf|to     |114  |5   |
|2303.10130v5.pdf|the    |374  |1   |
|2303.10130v5.pdf|and    |360  |2   |
|2303.10130v5.pdf|of     |270  |3   |
|2303.10130v5.pdf|to     |220  |4   |
|2303.10130v5.pdf|in     |154  |5   |
+----------------+-------+-----+----+



In [10]:
term = "attention"  # término a buscar
df_term_counts = df_palabras_pdf.filter(col("palabra").rlike(term))\
                                .groupBy("documento").count()
df_term_counts.show()

+----------------+-----+
|       documento|count|
+----------------+-----+
|2303.10130v5.pdf|    2|
|1706.03762v7.pdf|   97|
+----------------+-----+

